In [1]:
%matplotlib inline

import os
import numpy as np
import matplotlib.pyplot as plt

import matplotlib.animation as animation
import scipy.io as sio

default_dir = os.path.dirname(os.getcwd())

os.chdir(default_dir)

from scripts import celegans_dynome as c_dynome
from scripts import celegans_utilities as c_util
from scripts import sys_paths as paths

In [2]:
neurons_list = c_dynome.neurons_list

In [13]:
### start simulation


mylist=[] # list of probability
singular_values=[] # stores percentage of all singular values

# for every neuron in the network
for i in range(0,279):
    # clear the mask
    input_Mask = np.zeros(c_dynome.N)
    ablation_Mask = np.ones(c_dynome.N, dtype = 'bool')
    # inject input of 20k into the i-th neuron
    input_Mask[i]=0.2
    # run the network for 15 seconds
    result_dict = c_dynome.run_network(0, 15, 0.01, input_Mask, ablation_Mask)
    # get the results
    nsteps = result_dict['steps']
    Traj = result_dict['trajectory_mat']
    Vth = result_dict['v_threshold']
    VsubVth = np.subtract(Traj, Vth).transpose()
    # perform SVD on the resulting matrix
    U, s, Z = np.linalg.svd(VsubVth[:,100:], full_matrices=False)
    # compute the conditional probabilities
    
    percentage=np.divide(np.power(s, 2), np.sum(np.power(s, 2)))[:]
    weighted=[]
    result=np.zeros(279)
    for j in range(279):
        weighted=[abs(x)*percentage[j] for x in U[:,j]]
        result=[x+y for x,y in zip(result, weighted)] 
 
    prob=result/result[i]
    mylist.extend(prob)
    singular_values.extend(percentage)

output = open('weighted_sum.pkl', 'wb')
pickle.dump(mylist, output)
output.close() 

output=open('singular_values.pkl','wb')
pickle.dump(singular_values, output)
output.close()


cpt=np.reshape(mylist,(279,279))
import pandas as pd
cpt_new=pd.DataFrame(cpt)
cpt_new.to_csv('cpt_new.csv')

NameError: name 'pickle' is not defined

In [12]:
(np.tile(Vth, (nsteps, 1)))c.shape

(2253001, 279)